In [ ]:
import numpy as np
import pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense, Bidirectional, LSTM, Dropout, GRU

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# 1. 데이터 불러오기

train = pd.read_csv('/kaggle/input/nlpsci/train.csv')
test = pd.read_csv('/kaggle/input/nlpsci/test.csv')
sub = pd.read_csv('/kaggle/input/nlpsci/sample_submission.csv')

In [ ]:
for i in range(len(train['ABSTRACT'])):
    train['ABSTRACT'][i].replace('\n', ' ')
    
for i in range(len(test['ABSTRACT'])):
    test['ABSTRACT'][i].replace('\n', ' ')

In [ ]:
train['data'] = train['TITLE']+train['ABSTRACT']
test['data'] = test['TITLE']+test['ABSTRACT']

In [ ]:
train

In [ ]:
# 2. 데이터 전처리
train['data'] = train['TITLE']+train['ABSTRACT']
test['data'] = test['TITLE']+test['ABSTRACT']

all_data = pd.concat([train['data'], test['data']])
all_data

In [ ]:
# 3. Tokenizer

tk = Tokenizer()
tk.fit_on_texts(all_data)

In [ ]:
# 4. texts_to_sequence

tseq = tk.texts_to_sequences(all_data)

In [ ]:
# 5. pad_sequence

tpad = pad_sequences(tseq, maxlen = 80, padding = 'post')
tpad

In [ ]:
# 6. train, test split

train0 = tpad[:15972]
test0 = tpad[15972:]

train_y = train['label']
train_y = np.array(train_y)
np.unique(train_y, return_counts = True)

In [ ]:
# 7. set model
# 임베딩 사이즈는 보통 100~300이라 함. 이보다 작으면 의미를 뽑아내기 힘들어지거나 무용일 수 있음.
# 중요한 것은 network architecture, algorithm & dataset size.

model = Sequential()
model.add(Embedding(len(tk.word_index)+1, 300, input_length = 80))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.2))

model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(32, activation = 'relu'))
model.add(Dropout(0.5))

# model.add(Dense(16, activation = 'relu'))
# model.add(Dropout(0.5))

model.add(Dense(6, activation = 'softmax'))
model.summary()

In [ ]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = 'acc')
hist = model.fit(train0, train_y, epochs = 3, batch_size = 512)

In [ ]:
result = model.predict(test0)

In [ ]:
Result=result.argmax(1)
sub['label']=pd.DataFrame(Result)
sub.to_csv('submission.csv', index=False)